In [1]:
from library_dicom.dicom_processor.model.reader.Instance_RTSS import Instance_RTSS

from library_dicom.rtss_processor.model.MaskBuilder_RTSS import MaskBuilder_RTSS
from library_dicom.dicom_processor.model.Series import Series

import matplotlib.pyplot as plt
import numpy as np

### RTSTRUCT file -> Mask 

In [ ]:
rtss_path = 

In [ ]:
mask_builder_rtss = MaskBuilder_RTSS(rtss_path, CT_path)

In [ ]:
from time import process_time
t1_start = process_time()
mask_4D = mask_builder_rtss.rtss_to_4D_mask()
t1_stop = process_time() 
print("diff début fin :", t1_stop-t1_start)  

In [ ]:
mask_builder_rtss.number_of_roi

In [ ]:
plt.imshow(mask_4D[:,:,308,0]) #0 pour ROI 1
plt.show()
#3Dslicer <= 309 slice |  308 ième => Python 

### Mask -> RTSTRUCT file 


In [ ]:
from library_dicom.rtss_processor.model.RTSS_Writer import RTSS_Writer
import pydicom

In [ ]:
dataset = {}
for i in range(1,11):
    data = {}
    data['ROIName'] = str(i)
    data['ROIGenerationAlgorithm'] = 'SEMIAUTOMATIC'
    dataset[i] = data
print(dataset)    

In [ ]:
new_rtss = RTSS_Writer(mask_4D, CT_path, dataset)


In [ ]:
directory = r'C:\Users\wendy\Documents\STAGE\DicomRT\201802409 BARAT GALPIN ELIANE\103893708 Tepscan FDG\new_rtss'
filename = r'test2.dcm'

In [ ]:
new_rtss.save_file(filename, directory)

In [ ]:
### Test RTSTRUCT CT ROI to PET ROI 

In [10]:
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT

ct_path = '/media/oncopole/DD 2To/Jakob/Example Data/1.2.826.0.1.3417726.3.1622984591/1.2.826.0.1.3417726.3.458434.20201008132231363'
pet_path = '/media/oncopole/DD 2To/Jakob/Example Data/1.2.826.0.1.3417726.3.1622984591/1.2.826.0.1.3417726.3.30343.20201008132234488'

rt_path = '/media/oncopole/DD 2To/Jakob/Example Data/1.2.826.0.1.3417726.3.1622984591/1.2.826.0.1.3417726.3.714187.20201008132226221/1.2.826.0.1.3417726.3.280294.20201008132226252.dcm'

In [3]:
serie_pet_object = SeriesPT(pet_path)
serie_ct_object = SeriesCT(ct_path)
instance_pet = serie_pet_object.get_instances_ordered()
instance_ct = serie_ct_object.get_instances_ordered()

In [8]:
origin_pet = instance_pet[0].get_instance_tags()['ImagePosition']
origin_ct = instance_ct[0].get_instance_tags()['ImagePosition']
print(origin_pet)
print(origin_ct)

delta = []
for i in range(3):
    origin_pet[i] = float(origin_pet[i])
    origin_ct[i] = float(origin_ct[i])

    delta.append(origin_ct[i] - origin_pet[i])
print(origin_pet)
print(origin_ct)
print("delta : ", delta)

["-49.68300975", "-55.30265248", "-57.10406392"]
["-50.293084", "-50.293084", "-65.307512"]
[-49.68300975, -55.30265248, -57.10406392]
[-50.293084, -50.293084, -65.307512]
delta :  [-0.6100742500000038, 5.009568479999999, -8.203448080000001]


In [39]:
rtss_object = Instance_RTSS(rt_path)
rtss_object.get_number_of_roi()
#ATTENTION : Instance RTSS, ROI qui commence a 1 et non à 0. A décaler de 1

index = [14, 25]
contour = []
for i in index:
    contour.append(rtss_object.get_contour_data(i))
    
for roi in contour : 
    for slice in roi : 
        for i in range(len(slice)):
            slice[i] = float(slice[i])

In [40]:
contour[0][0]

[32.968744, -6.0897680000209, -38.218544, 32.968744, -5.8797760000209, -38.218544, 32.54876, -5.8797760000209, -38.218544, 32.54876, -5.6697840000209, -38.218544, 32.128776, -5.6697840000209, -38.218544, 32.128776, -5.4597920000209, -38.218544, 31.708792, -5.4597920000209, -38.218544, 31.708792, -5.2498000000209, -38.218544, 31.4988, -5.2498000000209, -38.218544, 31.4988, -5.0398080000209, -38.218544, 31.288808, -5.0398080000209, -38.218544, 31.288808, -4.8298160000209, -38.218544, 31.078816, -4.8298160000209, -38.218544, 31.078816, -4.4098320000209, -38.218544, 30.868824, -4.4098320000209, -38.218544, 30.868824, -4.1998400000209, -38.218544, 30.658832, -4.1998400000209, -38.218544, 30.658832, -3.3598720000209, -38.218544, 30.868824, -3.3598720000209, -38.218544, 30.868824, -3.1498800000209, -38.218544, 31.288808, -3.1498800000209, -38.218544, 31.288808, -2.9398880000209, -38.218544, 32.968744, -2.9398880000209, -38.218544, 32.968744, -3.1498800000209, -38.218544, 33.178736, -3.1498800

In [41]:
#Tumeur 1.2L
print("slices Tumor 1.2L : ", len(contour[0]))
print("slices Tumor 2.2L : ", len(contour[1]))

slices Tumor 1.2L :  47
slices Tumor 2.2L :  50


In [42]:
#change coordonate 
#On a les coordonnées CT 
#On veut les transformer en coordonées PT 
#coord_ct - delta = coord_pt
contour_pet = contour
for roi in contour_pet : 
    for slice in roi : 
        for i in range(len(slice)):
            slice[i] = float(slice[i])
            if i%3 == 0 :
                slice[i] = slice[i] - delta[0]

            elif i%3 == 1 : 
                slice[i] = slice[i] -  delta[1]

            elif i%3 == 2 : 
                slice[i] = slice[i] - delta[2]

In [43]:
contour_pet[0][0]

[33.578818250000005, -11.099336480020899, -30.01509592, 33.578818250000005, -10.889344480020899, -30.01509592, 33.158834250000005, -10.889344480020899, -30.01509592, 33.158834250000005, -10.6793524800209, -30.01509592, 32.738850250000006, -10.6793524800209, -30.01509592, 32.738850250000006, -10.4693604800209, -30.01509592, 32.31886625, -10.4693604800209, -30.01509592, 32.31886625, -10.2593684800209, -30.01509592, 32.10887425, -10.2593684800209, -30.01509592, 32.10887425, -10.049376480020898, -30.01509592, 31.898882250000003, -10.049376480020898, -30.01509592, 31.898882250000003, -9.839384480020898, -30.01509592, 31.688890250000004, -9.839384480020898, -30.01509592, 31.688890250000004, -9.419400480020899, -30.01509592, 31.478898250000004, -9.419400480020899, -30.01509592, 31.478898250000004, -9.209408480020898, -30.01509592, 31.268906250000004, -9.209408480020898, -30.01509592, 31.268906250000004, -8.369440480020899, -30.01509592, 31.478898250000004, -8.369440480020899, -30.01509592, 31